# Cell 1: Environment Setup

In [1]:
# Install required packages (run only if not already installed)
# !pip install langchain langchain-groq langchain-community python-dotenv

from dotenv import load_dotenv
import os
import re

# Load environment variables from .env
load_dotenv()


True

# Cell 2: Initialize LLM (Groq LLaMA 3.1)

In [2]:
from langchain_groq import ChatGroq

# Initialize the LLM
llm = ChatGroq(
    temperature=0.65,
    model_name="llama-3.1-8b-instant",
    max_tokens=1500,
    top_p=0.9,
    frequency_penalty=0.2,
    presence_penalty=0.15
)


d:\OneDrive\Desktop\Ai Assignment\aienv\Lib\site-packages\langchain_groq\chat_models.py:364: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(
d:\OneDrive\Desktop\Ai Assignment\aienv\Lib\site-packages\langchain_groq\chat_models.py:364: UserWarning: WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
  warnings.warn(
d:\OneDrive\Desktop\Ai Assignment\aienv\Lib\site-packages\langchain_groq\chat_models.py:364: UserWarning: WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.
  warnings.warn(


# Cell 3: Define External Tools (Wikipedia & DuckDuckGo)

In [3]:
from langchain.agents import Tool
from langchain.utilities import WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper

# Define external tools
wiki_tool = Tool(
    name="Wikipedia",
    func=WikipediaAPIWrapper().run,
    description="Searches Wikipedia for background and historical information on the topic."
)

ddg_tool = Tool(
    name="DuckDuckGo",
    func=DuckDuckGoSearchAPIWrapper().run,
    description="Searches DuckDuckGo for current and wide-ranging information."
)


# Cell 4: Define Prompt Templates

In [4]:
from langchain.prompts import PromptTemplate

# Title prompt
heading_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a compelling and SEO-friendly blog title for the following topic:\n\nTopic: {topic}"
)

# Research prompt
research_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""You are a research assistant. Use the tools provided to gather a detailed summary about the following blog topic:

Topic: {topic}

Summarize the findings in a concise but informative paragraph."""
)

# Summary prompt
summary_prompt = PromptTemplate(
    input_variables=["research"],
    template="Write a concise and insightful summary based on the following research:\n\n{research}"
)

# Final blog prompt
blog_prompt = PromptTemplate(
    input_variables=["heading", "research", "summary"],
    template="""
# {heading}

## Introduction

Write an engaging and informative introduction that sets the context for the blog topic. Use the following research data to highlight the importance, relevance, and background of the subject. Make it inviting and informative to hook the reader.

Research Data:
{research}

---

## Detailed Insights and Analysis

Organize the main body of the blog into well-defined sections with clear subheadings. Use bullet points, numbered lists, or tables where appropriate to present data effectively. Explore the topic from multiple perspectives, covering:

- Key facts and statistics derived from the research
- Trends and recent developments relevant to the topic
- Challenges or opportunities associated with the subject
- Examples or case studies to illustrate points
- Any notable regional, demographic, or sector-specific insights
- Some statistics to support the points
- Must be minimum 500 words

Make the content rich and insightful, ensuring each section adds clear value to the reader.

---

## Practical Recommendations and Tips

Based on the research, provide actionable advice, tips, or suggestions relevant to readers interested in the topic. This can include:

- Best practices or strategies
- Resources or tools worth exploring
- Common pitfalls to avoid
- Future outlook or things to watch for

---

## Conclusion

Summarize the key takeaways from the blog in a concise and impactful way. Reinforce why the topic matters and encourage readers to engage further or take action.

Summary:
{summary}
"""
)


# Cell 5: Create Chains Using LLM

In [5]:
from langchain.chains import LLMChain

# Define LLM Chains
heading_chain = LLMChain(llm=llm, prompt=heading_prompt)
research_chain = LLMChain(llm=llm, prompt=research_prompt)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
blog_chain = LLMChain(llm=llm, prompt=blog_prompt)


C:\Users\aruna\AppData\Local\Temp\ipykernel_26424\3045004164.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  heading_chain = LLMChain(llm=llm, prompt=heading_prompt)


# Cell 6: Initialize Agent With Tools

In [6]:
from langchain.agents import initialize_agent, AgentType

# Create a multi-tool agent with both Wikipedia and DuckDuckGo
multi_tool_agent = initialize_agent(
    tools=[wiki_tool, ddg_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


C:\Users\aruna\AppData\Local\Temp\ipykernel_26424\1012059659.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  multi_tool_agent = initialize_agent(


# Cell 7: Markdown Cleaner Function (Optional)

In [7]:
def strip_markdown(md_text):
    """Remove Markdown formatting for terminal display."""
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', md_text)
    text = re.sub(r'^#+\s+', '', text, flags=re.MULTILINE)
    text = re.sub(r'!\[.*?\]\(.*?\)', '', text)
    text = re.sub(r'\[(.*?)\]\(.*?\)', r'\1', text)  # links
    text = re.sub(r'[-*]\s+', '• ', text)
    return text.strip()


# Cell 8: Blog Generation Function

In [8]:
def generate_blog(topic):
    print("\n🔍 Generating title...")
    heading = heading_chain.run(topic)

    print("\n📚 Gathering research from tools...")
    research = multi_tool_agent.run(topic)

    print("\n✍️ Summarizing research...")
    summary = summary_chain.run(research)

    print("\n📰 Generating final blog in Markdown format...\n")
    blog_content = blog_chain.run({
        "heading": heading,
        "research": research,
        "summary": summary
    })

    print(strip_markdown(blog_content))  # Print to terminal


# Cell 9: Run the Blog Generator

In [9]:
# Run interactively
user_topic = input("Enter a topic for your blog: ")
generate_blog(user_topic)



🔍 Generating title...


C:\Users\aruna\AppData\Local\Temp\ipykernel_26424\1318636545.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  heading = heading_chain.run(topic)



📚 Gathering research from tools...


> Entering new AgentExecutor chain...
Thought: To answer the question about tourism in India, I need to gather information about the topic.
Action: Wikipedia
Action Input: tourism in India
Observation: Page: Tourism in India
Summary: Tourism in India is 4.6% of the country's gross domestic product (GDP). Unlike other sectors, tourism is not a priority sector for the Government of India. The World Travel and Tourism Council calculated that tourism generated ₹13.2 lakh crore (US$160 billion) or 5.8% of India's GDP and supported 32.1 million jobs in 2021. Even though, these numbers were lower than the pre-pandemic figures; the country's economy witnessed a significant growth in 2021 after the massive downturn during 2020. The sector is predicted to grow at an annual rate of 7.8% to ₹33.8 lakh crore (US$400 billion) by 2031 (7.2% of GDP). India has established itself as the 5th largest global travel healthcare destination with an estimated market size 